# Finding energy states of the Discretized 1D Particle in a Box using SSVQE

#### In this Jupyter notebook, we use the SSVQE algorithm to find eigenenergies/eigenstates of a particle in an infinite potential well. 
##### Credit to Yu Jun Shen (www.github.com/GryphonTorch) for initial inspiration.

#### 0. Imports + Logger Setup

In [1]:
from qiskit import * 
import numpy as np
import pandas as pd
import itertools
import functools
from enum import Enum
import logging 

open('SSVQE.log', 'w').close()
log1 = logging.getLogger('')
log1.addHandler(logging.FileHandler('SSVQE.log'))
log1.setLevel(25) # above DEBUG

KeyboardInterrupt: 

#### 1. Global Variables 

##### Hyperparameters: MODIFY THESE.

In [ ]:
''' The number of qubits, effectively how discretized the well is. Must be >= 2 since ansatzes have 2-qubit gates. '''
n = 2

''' The chosen ansatz ids (ids are at the Start of Section 3).'''
ANSATZES = [1, 2, 3]

''' The number of iterations of the ansatz in the (SS)VQE circuit. '''
ITERS = 2

''' The number of measurements to take. More measurements = Slower, More Accurate.'''
SHOTS = 1024

''' Simultaneously find 0 through k-1 th energy states using SSVQE. If k=1, SSVQE becomes just VQE.'''
K = 2

'''Initial Parameters of the Ansatzes. '''
PARAMS = np.random.uniform(0, np.pi, size=10000)

'''Optimizer Iterations. '''
iterations = 1000

'''The Hamiltonian, a Hermitian Operator acting on N=2^n dimensional Complex Hilbert Space.
This will be initialized for the 1D particle in a box case in (2). Feel free to define your own 
Hamiltonian as well. '''
H = np.zeros((2 ** n, 2 ** n))

''' Weight function for SSVQE - any monotonically decreasing, non-negative function of i (i < K) works.'''
weight_name = "SSVQE_curved_2_qubits"
def get_weights(i):
  return K - (0.5 * i)
 #  return (K ** .75) - (i ** .75)

##### Hyperparameters: DO NOT MODIFY.

In [ ]:
N = 2 ** n # number of states
paulis = {"x": np.array([[0, 1],  [ 1, 0]], dtype=np.complex128),
    "y": np.array([[0, -1j],[1j, 0]], dtype=np.complex128),
    "z": np.array([[1, 0],  [0, -1]], dtype=np.complex128),
    "id": np.array([[1, 0],  [ 0, 1]], dtype=np.complex128),}

##### 2. Hamiltonian of the Discretized 1D Well. Feel free to change for your Hamiltonian of Interest.

##### Get the Hamiltonian

In [ ]:
dx = 1.0 / float(N - 1)   
a = 1 / float(2 * dx * dx)
H = np.zeros((N, N))  + 2 * a * np.identity(N)
for i in range(N - 1):
  H[i, i + 1] -= a
for i in range(1, N):
  H[i, i - 1] -= a

##### Hamiltonian Decomposition Helper Functions

In [ ]:
def HS(M1, M2):
    """Hilbert-Schmidt-Product of two matrices M1, M2"""
    return (np.dot(M1.conjugate().transpose(), M2)).trace()

def decompose(H):
    """Decompose Hamiltonian H into Pauli matrices"""
    from numpy import kron
    decomposition = dict()
    for pauli_string in itertools.product(paulis.items(), repeat=n):
      factor = tuple([x[0] for x in pauli_string])
      coefficient = np.real(1/N * HS(functools.reduce(lambda x, y: np.kron(x, y), [x[1] for x in pauli_string]), H))
      if np.abs(coefficient) > 0:
        decomposition[factor] = coefficient
    return decomposition

#### 3. Ansatzes: CAN ADD OR MODIFY.

##### Ansatz Information

In [ ]:
class Ansatz(Enum):
  Hardware_Efficient_Ansatz = 0
  CNOT_Ansatz = 1
  Adiabatic_Ansatz = 2
  NewCirc1 = 3
  NewCirc2 = 4
  NewCirc3 = 5
  NewCirc4 = 6
  NewCirc5 = 7
  NewCirc6 = 8
  NewCirc7 = 9
  NewCirc8 = 10
  NewCirc9 = 11
  NewCirc10 = 12
  NewCirc11 = 13
  NewCirc12 = 14
  NewCirc13 = 15
  NewCirc14 = 16
  NewCirc15 = 17
  NewCirc16 = 18
  NewCirc17 = 19
  NewCirc18 = 20
  NewCirc19 = 21
  
pcount = { Ansatz.Hardware_Efficient_Ansatz: 2 * n * ITERS,
        Ansatz.CNOT_Ansatz: 2 * n * ITERS,
        Ansatz.Adiabatic_Ansatz: n * (ITERS + 1),
        Ansatz.NewCirc1: 2 * n * ITERS,
        Ansatz.NewCirc2: 2 * n * ITERS,
        Ansatz.NewCirc3: 2 * n * ITERS + (n - 1) * ITERS,
        Ansatz.NewCirc4: 2 * n * ITERS + (n - 1) * ITERS,
        Ansatz.NewCirc5: 2 * n * ITERS + (n - 1) * n * ITERS,
        Ansatz.NewCirc6: 2 * n * ITERS + (n - 1) * n * ITERS,
        Ansatz.NewCirc7: 4 * n * ITERS + (n - 1) * ITERS,
        Ansatz.NewCirc8: 4 * n * ITERS + (n - 1) * ITERS,
        Ansatz.NewCirc9: n * ITERS,
        Ansatz.NewCirc10: n * (ITERS + 1),
        Ansatz.NewCirc11: 3 * n * ITERS, # only for 4 qubit case 
        Ansatz.NewCirc12: 3 * n * ITERS, # only for 4 qubit case 
        Ansatz.NewCirc13: 4 * n * ITERS,
        Ansatz.NewCirc14: 4 * n * ITERS,
        Ansatz.NewCirc15: 2 * n * ITERS,
        Ansatz.NewCirc16: 2 * n * ITERS + (n - 1) * ITERS,
        Ansatz.NewCirc17: 2 * n * ITERS + (n - 1) * ITERS,
        Ansatz.NewCirc18: 3 * n * ITERS,
        Ansatz.NewCirc19: 3 * n * ITERS,
}

##### First Ansatz Choices

In [ ]:
# Hardware Efficient Ansatz with ITERS iterations of the skeleton
def Hardware_Efficient_Ansatz(params):
  assert len(params) == pcount[Ansatz.Hardware_Efficient_Ansatz] 
  qc = QuantumCircuit(n, n)
  p = 0 # parameter count
  for _ in range(ITERS):
    for i in range(n):
      qc.ry(params[p], i) 
      p += 1
      qc.rz(params[p], i)
      p += 1
    for i in range(n - 1):
      qc.rzz(np.pi/2, i, i + 1)
  return qc

In [ ]:
def CNOT_Ansatz(params):
  assert len(params) == pcount[Ansatz.CNOT_Ansatz]   
  qc = QuantumCircuit(n, n)
  p = 0 # parameter count
  for _ in range(ITERS):
    for i in range(n):
      qc.ry(params[p], i) 
      p += 1
      qc.rz(params[p], i)
      p += 1
    for i in range(0, n - 1):
      for j in range(i + 1, n):
        qc.cx(i, j)
  return qc

In [ ]:
def Adiabatic_Ansatz(params):
  assert len(params) == pcount[Ansatz.Adiabatic_Ansatz]   
  qc = QuantumCircuit(n, n)
  p = 0 # parameter count
  for i in range(n):
    qc.ry(params[p], i)
    p += 1
  for _ in range(ITERS):
    for i in range(0, n - 1):
        qc.cx(i, i + 1)
    for i in range(n):
      qc.ry(params[p], i) 
      p += 1
    for i in range(1, n - 1):
      qc.x(i)
      
  return qc

#####  Ansatzes From "Expressibility and Entangling Capability of Parametrized Quantum Circuits"

In [ ]:
def NewCirc1(params):
    assert len(params) == pcount[Ansatz.NewCirc1] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
    return qc

In [ ]:
def NewCirc2(params):
    assert len(params) == pcount[Ansatz.NewCirc2] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
            if i != 0:
                qc.cx(i, i - 1)
    return qc

In [ ]:
def NewCirc3(params):
    assert len(params) == pcount[Ansatz.NewCirc3] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
            if i != 0:
                qc.crz(params[p], i, i - 1)
                p += 1
    return qc

In [ ]:
def NewCirc4(params):
    assert len(params) == pcount[Ansatz.NewCirc4] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
            if i != 0:
                qc.crx(params[p], i, i - 1)
                p += 1
    return qc

In [ ]:
def NewCirc5(params):
    assert len(params) == pcount[Ansatz.NewCirc5] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n - 1, -1, -1):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
            for j in range(n - 1, -1, -1):
                if i != j:
                    qc.crz(params[p], i, j)
                    p += 1
    return qc

In [ ]:
def NewCirc6(params):
    assert len(params) == pcount[Ansatz.NewCirc6] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n - 1, -1, -1):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
            for j in range(n - 1, -1, -1):
                if i != j:
                    qc.crx(params[p], i, j)
                    p += 1
    return qc

In [ ]:
def NewCirc7(params):
    assert len(params) == pcount[Ansatz.NewCirc7] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
        for j in range(1, n, 2):
            qc.crz(params[p], j, j - 1)
            p += 1
        for i in range(n):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
        for j in range(2, n, 2):
            qc.crz(params[p], j, j - 1)
            p += 1
    return qc

In [ ]:
def NewCirc8(params):
    assert len(params) == pcount[Ansatz.NewCirc8] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
        for j in range(1, n, 2):
            qc.crx(params[p], j, j - 1)
            p += 1
        for i in range(n):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
        for j in range(2, n, 2):
            qc.crx(params[p], j, j - 1)
            p += 1
    return qc

In [ ]:
def NewCirc9(params):
    assert len(params) == pcount[Ansatz.NewCirc9] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n - 1, -1, -1):
            qc.h(i)
            if i != 0:
                qc.cz(i, i - 1)

            qc.rx(params[p], i)
            p += 1
    return qc

In [ ]:
def NewCirc10(params):
    assert len(params) == pcount[Ansatz.NewCirc10] 
    qc = QuantumCircuit(n, n)
    p = 0
    for j in range(n):
        qc.ry(params[p], j)
        p += 1

    for _ in range(ITERS):
        for i in range(n - 1, -1, -1):
            if i != 0:
                qc.cz(i, i - 1)
            else:
                qc.cz(i, n - 1)
            qc.rx(params[p], i)
            p += 1
    return qc

In [ ]:
def NewCirc11(params): #ASSUME n = 4
    assert len(params) == pcount[Ansatz.NewCirc11] 
    qc = QuantumCircuit(4, 4)
    p = 0
    for _ in range(ITERS):
        for i in range(4):
            qc.ry(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
        qc.cx(1, 0)
        qc.cx(3, 2)
        qc.ry(params[p], 1)
        p += 1
        qc.ry(params[p], 2)
        p += 1
        qc.rz(params[p], 1)
        p += 1
        qc.rz(params[p], 2)
        p += 1
        qc.cx(2, 1)
            
    return qc

In [ ]:
def NewCirc12(params): #ASSUME n = 4
    assert len(params) == pcount[Ansatz.NewCirc12] 
    qc = QuantumCircuit(4, 4)
    p = 0
    for _ in range(ITERS):
        for i in range(4):
            qc.ry(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
        qc.cz(0, 1)
        qc.cz(2, 3)
        qc.ry(params[p], 1)
        p += 1
        qc.ry(params[p], 2)
        p += 1
        qc.rz(params[p], 1)
        p += 1
        qc.rz(params[p], 2)
        p += 1
        qc.cz(1, 2)
            
    return qc

In [ ]:
def NewCirc13(params):
    assert len(params) == pcount[Ansatz.NewCirc13] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n - 1, -1 , -1):
            qc.ry(params[p], i)
            p += 1
            if i != n - 1:
                qc.crz(params[p], i, i + 1)
                p += 1
            else:
                qc.crz(params[p], i, 0)
                p += 1
            qc.ry(params[p], i)
            p += 1
            if i != 0:
                qc.crz(params[p], i, i - 1)
                p += 1
            else:
                qc.crz(params[p], i, n - 1)
                p += 1
    return qc

In [ ]:
def NewCirc14(params):
    assert len(params) == pcount[Ansatz.NewCirc14] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n - 1, -1 , -1):
            qc.ry(params[p], i)
            p += 1
            if i != n - 1:
                qc.crx(params[p], i, i + 1)
                p += 1
            else:
                qc.crx(params[p], i, 0)
                p += 1
            qc.ry(params[p], i)
            p += 1
            if i != 0:
                qc.crx(params[p], i, i - 1)
                p += 1
            else:
                qc.crx(params[p], i, n - 1)
                p += 1
    return qc

In [ ]:
def NewCirc15(params):
    assert len(params) == pcount[Ansatz.NewCirc15] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n - 1, -1 , -1):
            qc.ry(params[p], i)
            p += 1
            if i != n - 1:
                qc.cx(i, i + 1)
            else:
                qc.cx(i, 0)
            qc.ry(params[p], i)
            p += 1
            if i != 0:
                qc.cx(i, i - 1)
            else:
                qc.cx(i, n - 1)
    return qc

In [ ]:
def NewCirc16(params):
    assert len(params) == pcount[Ansatz.NewCirc16] 
    return NewCirc3(params)

In [ ]:
def NewCirc17(params):
    assert len(params) == pcount[Ansatz.NewCirc17] 
    return NewCirc4(params)

In [ ]:
def NewCirc18(params):
    assert len(params) == pcount[Ansatz.NewCirc18] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n - 1, -1 , -1):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
            if i != n - 1:
                qc.crz(params[p], i, i + 1)
                p += 1
            else:
                qc.crz(params[p], i, 0)
                p += 1
    return qc

In [ ]:
def NewCirc19(params):
    assert len(params) == pcount[Ansatz.NewCirc19] 
    qc = QuantumCircuit(n, n)
    p = 0
    for _ in range(ITERS):
        for i in range(n - 1, -1 , -1):
            qc.rx(params[p], i)
            p += 1
            qc.rz(params[p], i)
            p += 1
            if i != n - 1:
                qc.crx(params[p], i, i + 1)
                p += 1
            else:
                qc.crx(params[p], i, 0)
                p += 1
    return qc

##### Converts Ansatz to Function

In [ ]:
ansatz_to_fn = {Ansatz.Hardware_Efficient_Ansatz: Hardware_Efficient_Ansatz, 
        Ansatz.CNOT_Ansatz: CNOT_Ansatz,
        Ansatz.Adiabatic_Ansatz: Adiabatic_Ansatz,
        Ansatz.NewCirc1: NewCirc1,
        Ansatz.NewCirc2: NewCirc2,
        Ansatz.NewCirc3: NewCirc3,
        Ansatz.NewCirc4: NewCirc4,
        Ansatz.NewCirc5: NewCirc5,
        Ansatz.NewCirc6: NewCirc6,
        Ansatz.NewCirc7: NewCirc7,
        Ansatz.NewCirc8: NewCirc8,
        Ansatz.NewCirc9: NewCirc9,
        Ansatz.NewCirc10: NewCirc10,
        Ansatz.NewCirc11: NewCirc11, # only for 4 qubit case 
        Ansatz.NewCirc12: NewCirc12, # only for 4 qubit case 
        Ansatz.NewCirc13: NewCirc13,
        Ansatz.NewCirc14: NewCirc14,
        Ansatz.NewCirc15: NewCirc15,
        Ansatz.NewCirc16: NewCirc16,
        Ansatz.NewCirc17: NewCirc17,
        Ansatz.NewCirc18: NewCirc18,
        Ansatz.NewCirc19: NewCirc19,
        }

#### 4. VQE + SSVQE: DO NOT MODIFY.

In [ ]:
decomposition_terms = decompose(H) # get decomposition terms

def all_ansatzes(params):
  ansatzes = dict()
  for tproducts, coeff in decomposition_terms.items():
    if np.abs(coeff) != 0:
      if tproducts.count("id") == n:
        qc = QuantumCircuit(1, 1) # circuit always measuring 1
        qc.measure(0, 0)
        ansatzes[tproducts] = (coeff, qc)
      else:
        circuit = ansatz_to_fn[ANSATZ](params)
        for i in range(n):
          pauli = tproducts[i]
          if pauli == "x":
            circuit.h(i)
          elif pauli == "y":
            circuit.sdg(i)
            circuit.h(i)
          if pauli != "id": # don't need to measure identity observables
            circuit.measure(i, i)
        ansatzes[tproducts] = (coeff, circuit)
    else:
      logging.error('Should not be attempting to compute ansatz when coefficient is zero.')
  return ansatzes

Now we have all the ansatzes to do VQE and their corresponding coefficients. NOTE: This VQE is  SSVQE with K=1, so we never use this function (only for me to practice). :

In [ ]:

# backend = Aer.get_backend("statevector_simulator")


def VQE(parameters):
  ansatzes = all_ansatzes(parameters)
  guess = 0
  for ansatz in ansatzes:
    t_qc = transpile(ansatzes[ansatz][1], backend)
    qobj = assemble(t_qc, shots=SHOTS)     
    result = backend.run(qobj).result()
    output = result.get_counts(ansatzes[ansatz][1])
    # calculate averaged measurement result (+1 iff even number of 1's, otw -1)
    avg_measurement = 0
    for res in output:
        if str(res).count("1") % 2 == 0:
          avg_measurement += output[res]
        else:
          avg_measurement -= output[res]
    guess += avg_measurement * ansatzes[ansatz][0]
  
  guess /= SHOTS
    
  return guess

In [ ]:
backend = Aer.get_backend("qasm_simulator")

def SSVQE(parameters):
  total_cost = 0
  for i in range(K):
    weight = get_weights(i) #  weight function 

    ############################################################################

    ## ANALYTIC APPROACH: if we're gonna simulate, instead lets just calculate ev directly ##
    # circuit = ansatz_to_fn[ANSATZ](parameters)
    # result = np.array(execute(circuit, backend).result().get_statevector())
    # resultH = np.conj(result).T
    # ev = resultH @ H @ result
    # ev = np.real(ev)
        
    # total_cost += ev * weight 

    ############################################################################

    ## EXPERIMENTAL APPROACH: Dd decomposition + minimize each basis state ##

    ansatzes = all_ansatzes(parameters)

    intermediate_cost = 0

    for ansatz in ansatzes: 
      if ansatzes[ansatz][1].depth() > 1: # hacky way of checking it's not just identity ansatz, a special case
        circuit = k_state(i).compose(ansatzes[ansatz][1])
      else:
        # quick and dirty +1 measurement on all id case
        circuit = QuantumCircuit(1)
        circuit.measure_all()
   
      result = execute(circuit, backend, shots=SHOTS).result()
      output = result.get_counts(circuit)
      # calculate averaged measurement result (+1 iff even number of 1's, otw -1)
      avg_measurement = 0
      for res in output:
          if str(res).count("1") % 2 == 0:
            avg_measurement += output[res]
          else:
            avg_measurement -= output[res]
      avg_measurement /= SHOTS
      
      if np.abs(avg_measurement) > 1:
        logging.error(f"Bad measurement: {avg_measurement}")

      intermediate_cost += (avg_measurement * ansatzes[ansatz][0])

    # print(f"E({i}):", np.round(intermediate_cost, 3), end= " ") # relevant for SSVQE
    total_cost += intermediate_cost * weight

  return total_cost

# return circuit outputting |k>.
def k_state(k):
  qc = QuantumCircuit(n, n)
  for i in range(n):
    if (k >> i) % 2 == 1:
      qc.x(i)
  return qc 

#### 5. Run the SPSA Optimization Procedure.

In [ ]:
from qiskit.algorithms.optimizers import SPSA

def SPSA_optimize(ansatz, iterations):
  global ANSATZ
  ANSATZ = ansatz
  info = [] 
  iter = 0
  def get_SPSA_info(a1,a2,a3,a4,a5):
    nonlocal iter 
    iter += 1

    # ADD COST
    info.append((iter, a2, a3))
    if iter % 10 == 0:
      logging.log(25, f"Iter:{iter}, Cost:{a3}")

  logging.log(25, f"Starting optimization with Ansatz={ANSATZ}")
  optimizer = SPSA(maxiter=iterations, blocking=True, allowed_increase=100, callback=get_SPSA_info)
  optimizer.optimize(pcount[ANSATZ], SSVQE, initial_point=PARAMS[:pcount[ANSATZ]])
  return info

#### 6. Analysis: ADD OR MODIFY

##### Analysis for VQE (not SSVQE)

In [ ]:
from os.path import exists
from datetime import datetime

e0 = np.min(np.linalg.eigvals(H)) # lowest eigenvalue for error checking

df = pd.DataFrame(columns=["Date/Time", "Hamiltonian", "Weight Name", "n qubits", "Ansatz Name", "Ansatz Id", "Ansatz Layers", 
"Optimizer", "Optimizer Iterations", "Theoretical e0", "Iters for e0 5-approx", "Iters for e0 2-approx", "Iters for e0 1.1-approx", "Iters for e0 1.01-approx",
"Energies for Lowest Cost Iteration", "Data for Lowest Cost Iteration", "Simulation Data"])

if exists("SSVQE_results.df"):
    df = pd.read_pickle("SSVQE_results.df")

for a in ANSATZES:

    if n == 4 or a not in [13, 14]: # these two ansatzes are 4-qubit specific

        results = SPSA_optimize(Ansatz(a), iterations)
        best = sorted(results, key=lambda x: x[2])[0] # lowest cost function
        a0 = dict()
        for i in range(K):
            circuit = k_state(i).compose(ansatz_to_fn[ANSATZ](best[1]))
            result = np.array(execute(circuit, Aer.get_backend("statevector_simulator")).result().get_statevector())
            resultH = np.conj(result).T
            a0[i] = np.real(resultH @ H @ result)

        a5, a2, a1d1, a1d01 = None, None, None, None
        for i in range(iterations):

            # COMPUTE THE EV(s)
            evs = dict()
            for j in range(K):
                circuit = k_state(j).compose(ansatz_to_fn[ANSATZ](results[i][1]))
                result = np.array(execute(circuit, Aer.get_backend("statevector_simulator")).result().get_statevector())
                resultH = np.conj(result).T
                evs[j] = np.real(resultH @ H @ result)

            # EV CAN'T BE TOO LOW 
            if min(evs.values()) < e0:
                logging.critical(f"FATAL: measurement outcome of {min(evs.values())} during iteration {i} violates variational principle, i.e. lowest possible measurement={e0}.")

            # GROUND STATE APPROX FACTORS
            factor = evs[0] / e0
            if not a5 and factor <= 5:
                a5 = i
            if not a2 and factor <= 2:
                a2 = i
            if not a1d1 and factor <= 1.1:
                a1d1 = i
            if not a1d01 and factor <= 1.01:
                a1d01 = i
                    
        df.loc[len(df.index)] = [datetime.now(), "1D Infinite Well", weight_name, n, Ansatz(a).name, a, ITERS, "SPSA", iterations, e0, a5, a2, a1d1, a1d01, a0, best, results]

    df.to_pickle("SSVQE_results.df") # save at intermediate steps so we can kill simulation even when only partially done 


##### More Analysis (TODO)